In [1]:
import h5py 
import numpy as np
import xarray as xr

# Flow Reconstruction of Nozzle Flow with heat transfer

We are trying to build a flow reconstruction model for the internal flow of a nozzle subjected to heat tranfer in the walls. The *low fidelity* dataset was generated with and in-house quasi-1D Euler solver (must place this code in another repository). While the *high fidelity* dataset was generated using the conjugate heat transfer finite volume solver [SU2](https://su2code.github.io/). 


Both datasets were storage in a standard .hdf5 file format, and can be loaded as 

In [2]:
low_fidelity_dataset = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_multirun_N200/Q1D.hdf5'
low_fidelity_vars_to_import = ['p.txt', 'T.txt', 'M.txt']

high_fidelity_dataset = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_multirun_N200/SU2_fluid.hdf5'
high_fidelity_vars_to_import = ['Pressure', 'Temperature', 'Mach', 'Heat_Flux']

with h5py.File(low_fidelity_dataset, 'r') as f:
    print('low fidelity dataset keys:')
    print(f.keys())
    
with h5py.File(high_fidelity_dataset, 'r') as f:
    print('high fidelity dataset keys:')
    print(f.keys())

low fidelity dataset keys:
<KeysViewHDF5 ['DoE', 'M.txt', 'T.txt', 'T0in', 'id', 'p.txt', 'p0in', 'thickness']>
high fidelity dataset keys:
<KeysViewHDF5 ['DoE', 'Heat_Flux', 'Mach', 'Pressure', 'T0in', 'Temperature', 'id', 'meshfile', 'p0in', 'thickness', 'wall_x', 'wall_y']>


In [13]:
lfds = xr.open_dataset(low_fidelity_dataset, chunks=-1)

lfds = lfds.swap_dims({
    'phony_dim_5': 'M',
    'phony_dim_6': 'id'
})

lfds = lfds.assign_coords({
    'M': ("M", lfds['M'].values),
    'id': ("id", lfds['id'].values),
    'T0in': ("id", lfds['T0in'].values), 
    'p0in': ("id", lfds['p0in'].values),
    'thickness': ("id", lfds['thickness'].values),
})
lfds 


<xarray.Dataset>
Dimensions:    (M: 211, id: 181)
Coordinates:
    T0in       (id) float64 587.1 818.7 668.9 561.2 ... 762.9 913.6 528.2 464.5
  * id         (id) int32 1 2 3 4 5 6 7 8 9 ... 190 191 192 193 194 195 198 199
    p0in       (id) float64 1.191e+06 1.191e+06 ... 1.191e+06 1.191e+06
    thickness  (id) float64 0.0074 0.0074 0.0074 0.0074 ... 0.0074 0.0074 0.0074
  * M          (M) int64 0 1 2 3 4 5 6 7 8 ... 203 204 205 206 207 208 209 210
Data variables:
    M.txt      (M, id) float64 dask.array<chunksize=(211, 181), meta=np.ndarray>
    T.txt      (M, id) float64 dask.array<chunksize=(211, 181), meta=np.ndarray>
    p.txt      (M, id) float64 dask.array<chunksize=(211, 181), meta=np.ndarray>

In [11]:

lfds.to_netcdf('lfds.nc')

In [17]:
hfds = xr.open_dataset(high_fidelity_dataset, chunks=-1)

hfds = hfds.swap_dims({
    'phony_dim_5': 'M',
    'phony_dim_6': 'id',
    'phony_dim_7': 'N',})

hfds = hfds.assign_coords({
    'T0in': ("id", hfds['T0in'].values), 
    'p0in': ("id", hfds['p0in'].values),
    'thickness': ("id", hfds['thickness'].values),
    'wall_x' : (["M","id"], hfds['wall_x'].values),
    'wall_y' : (["M","id"], hfds['wall_y'].values),
})

hfds

<xarray.Dataset>
Dimensions:      (M: 210, id: 181, N: 69300)
Coordinates:
    T0in         (id) float64 587.1 818.7 668.9 561.2 ... 913.6 528.2 464.5
  * id           (id) int32 1 2 3 4 5 6 7 8 ... 190 191 192 193 194 195 198 199
    p0in         (id) float64 1.191e+06 1.191e+06 ... 1.191e+06 1.191e+06
    thickness    (id) float64 0.0074 0.0074 0.0074 ... 0.0074 0.0074 0.0074
    wall_x       (M, id) float64 0.001245 0.001245 0.001245 ... 0.1504 0.1504
    wall_y       (M, id) float64 0.0628 0.0628 0.0628 ... 0.03726 0.03726
Dimensions without coordinates: M, N
Data variables:
    Heat_Flux    (M, id) float64 dask.array<chunksize=(210, 181), meta=np.ndarray>
    Mach         (N, id) float64 dask.array<chunksize=(69300, 181), meta=np.ndarray>
    Pressure     (N, id) float64 dask.array<chunksize=(69300, 181), meta=np.ndarray>
    Temperature  (N, id) float64 dask.array<chunksize=(69300, 181), meta=np.ndarray>
    meshfile     (id) object dask.array<chunksize=(181,), meta=np.ndarray>

In [7]:


hfds = hfds.assign_coords({
    'M': ("M", lfds['M'].values),
    'id': ("id", hfds['id'].values),
    'T0in': ("id", hfds['T0in'].values), 
    'p0in': ("id", hfds['p0in'].values),
    'thickness': ("id", hfds['thickness'].values),
    'wall_x' : (["M","id"], hfds['wall_x'].values),
    'wall_y' : (["M","id"], hfds['wall_y'].values),
})
hfds.to_netcdf('hfds.nc')

ValueError: conflicting sizes for dimension 'M': length 211 on 'M' and length 210 on {'M': 'Heat_Flux', 'id': 'Heat_Flux', 'N': 'Mach'}

In [ ]:
for i in range(1,10):
    hfds.sel(id=i)['Heat_Flux'][1:-1].plot()


# Order Reduction

In [ ]:
class dataHandler:
    def __init__(self):
        pass

    def set_dataset(self, dataset, index, variables):
        self.dataset = dataset
        self.index = index
        self.variables = variables

    def snapshot(self, 
        index:int=0, 
        variable:list=None):
        if variable is None:
            variable = self.variables

        #concatenation = xr.concat([self.dataset[var] for var in self.variables], dim=self.index)
        #selection = concatenation.isel(**{self.index: index})
        return np.concatenate([self.dataset.isel(**{self.index: index})[var] for var in  variable])

In [ ]:
dir(snapshot)

In [ ]:
lfds[['p.txt','T.txt','M.txt']].isel(id=0).to_array()


In [ ]:
dh = dataHandler()

dh.set_dataset(
    dataset = lfds, 
    index = 'id', 
    variables = ['M.txt', 'T.txt', 'p.txt']
    )


In [ ]:
xr.concat([lfds['M.txt'], lfds['T.txt'], lfds['p.txt']], dim='M')


In [ ]:
hfds

In [ ]:
xr.concat([hfds.isel(id=0)], dim='N')


In [ ]:
import dask.array as da

In [ ]:
lfds = xr.open_dataset('lfds.nc', chunks={'M':10,'id':10})
lfds

In [ ]:
lfds['M.txt'].to_dask_dataframe().to_dask_array(lengths=True)

In [ ]:
da.linalg.svd(lfds['M.txt'].to_dask_dataframe())